In [16]:
#imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def double_box_table(file, csv_name):
    #importing events dataframe
    events=pd.read_json(file)

    #importing team names dataframe
    teams=pd.read_json('teams.json')
    teams=teams[['name','wyId']]

    #Creating the shots dataframe for plot purporses, outside this function
    shots=events[(events['eventName']=='Shot')]
    goal=[]
    y=[]
    x=[]
    distance=[]
    for row in range(shots.shape[0]):
        tag=shots.iloc[row, 2]
        if tag==None or len(tag)==0:
            flag=0
        else:
            flag=max([item['id']==101 for item in tag])
        if len(shots.iloc[row,4])<2:
            y_pos=0
            x_pos=0
            dist=0
        else:
            y_pos=shots.iloc[row,4][0]['y']
            x_pos=shots.iloc[row,4][0]['x']
            dist=np.sqrt(np.square((x_pos-100)*1.05)+np.square((y_pos-50)*0.68))
        goal.append(flag)
        y.append(y_pos*0.68)
        x.append(x_pos*1.05)
        distance.append(dist)
    shots['goal']=goal
    shots['y']=y
    shots['x']=x
    shots['distance']=distance
    shots['box']=np.where((shots['x']>=87.5) & (shots['y']<=50.5) & (shots['y']>=17.5),1,0)
    shots=shots[['box', 'distance', 'goal']]
    shots.to_csv(csv_name+'.csv')

    #creating the results dataframe, which in the end will be the final league table
    events=events[(events['eventId']==9) | (events['eventName']=='Foul')]
    goal=[]
    y=[]
    x=[]
    distance=[]
    for row in range(events.shape[0]):
        tag=events.iloc[row, 2]
        if tag==None or len(tag)==0:
            flag=0
        else:
            flag=max([item['id']==101 for item in tag])
        if len(events.iloc[row,4])<2:
            y_pos=0
            x_pos=0
            dist=0
        else:
            y_pos=events.iloc[row,4][1]['y']
            x_pos=events.iloc[row,4][1]['x']
            dist=np.sqrt(np.square((x_pos-0)*1.05)+np.square((y_pos-50)*0.68))
        goal.append(flag)
        y.append(y_pos*0.68)
        x.append(x_pos*1.05)
        distance.append(dist)
    events['goal']=goal
    events['y']=y
    events['x']=x
    events['distance']=distance
    events['box']=np.where((events['x']<=12.5) & (events['y']<=50.5) & (events['y']>=17.5),1,0)
    events['box_goals']=np.where((events['goal']==1) & (events['box']==0), 2, np.where (events['goal']==1,1,0))

    #now, creating the actual league table

    results=events.copy()
    results=results.groupby(['matchId','teamId'])[['goal']].sum().reset_index()
    most_goals=results.groupby('matchId')[['goal']].max().reset_index()
    most_goals = most_goals.rename(columns={'goal': 'goal_max'})
    least_goals=results.groupby('matchId')[['goal']].min().reset_index()
    least_goals = least_goals.rename(columns={'goal': 'goal_min'})
    results=results.merge(most_goals, on='matchId')
    results=results.merge(least_goals, on='matchId')
    results['points']=np.where(results['goal']>results['goal_min'],0, np.where(results['goal']<results['goal_max'],3,1))
    results['goals_dif']=np.where(results['points']==1,0, np.where(results['points']==3,results['goal_max']-results['goal'],results['goal_min']-results['goal']))

    results=results.groupby('teamId')[['points', 'goals_dif']].sum().reset_index()

    results=results.merge(teams, left_on=['teamId'], right_on=['wyId']).drop(['teamId', 'wyId'], axis=1)
    actual_table=results.sort_values(by=['points', 'goals_dif'], ascending=False)

    #creating the modified league table

    results=events.copy()
    results['goal']=results['box_goals']
    results=results.groupby(['matchId','teamId'])[['goal']].sum().reset_index()
    most_goals=results.groupby('matchId')[['goal']].max().reset_index()
    most_goals = most_goals.rename(columns={'goal': 'goal_max'})
    least_goals=results.groupby('matchId')[['goal']].min().reset_index()
    least_goals = least_goals.rename(columns={'goal': 'goal_min'})
    results=results.merge(most_goals, on='matchId')
    results=results.merge(least_goals, on='matchId')
    results['points']=np.where(results['goal']>results['goal_min'],0, np.where(results['goal']<results['goal_max'],3,1))
    results['goals_dif']=np.where(results['points']==1,0, np.where(results['points']==3,results['goal_max']-results['goal'],results['goal_min']-results['goal']))

    results=results.groupby('teamId')[['points', 'goals_dif']].sum().reset_index()

    results=results.merge(teams, left_on=['teamId'], right_on=['wyId']).drop(['teamId', 'wyId'], axis=1)
    new_table=results.sort_values(by=['points', 'goals_dif'], ascending=False)

    return actual_table, new_table


In [17]:
eng_old, eng_new=double_box_table('events_England.json', 'eng_shots')

<ipython-input-16-a3ed5dc79a08>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal']=goal
<ipython-input-16-a3ed5dc79a08>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['y']=y
<ipython-input-16-a3ed5dc79a08>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [4]:
eng_old

,points,goals_dif,name
8,100,79,Manchester City
2,81,40,Manchester United
7,77,38,Tottenham Hotspur
3,75,46,Liverpool
1,70,24,Chelsea
0,63,23,Arsenal
15,54,-3,Burnley
6,49,-14,Everton
11,47,-4,Leicester City
4,44,-8,Newcastle United


In [5]:
eng_new

,points,goals_dif,name
8,102,98,Manchester City
2,83,63,Manchester United
3,80,64,Liverpool
7,76,54,Tottenham Hotspur
0,67,32,Arsenal
1,66,29,Chelsea
15,57,5,Burnley
12,49,-20,West Ham United
11,48,-10,Leicester City
6,48,-15,Everton


In [18]:
fra_old, fra_new=double_box_table('events_France.json', 'fra_shots')

<ipython-input-16-a3ed5dc79a08>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal']=goal
<ipython-input-16-a3ed5dc79a08>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['y']=y
<ipython-input-16-a3ed5dc79a08>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [7]:
fra_old

,points,goals_dif,name
1,93,79,PSG
19,80,40,Monaco
0,77,43,Olympique Lyonnais
3,77,33,Olympique Marseille
5,58,6,Rennes
4,55,5,Bordeaux
11,55,-3,Saint-\u00c9tienne
6,54,1,Nice
2,52,-5,Nantes
14,51,3,Montpellier


In [8]:
fra_new

,points,goals_dif,name
1,93,104,PSG
19,78,55,Monaco
0,76,64,Olympique Lyonnais
3,67,28,Olympique Marseille
6,61,6,Nice
5,59,10,Rennes
14,58,12,Montpellier
11,58,0,Saint-\u00c9tienne
4,55,6,Bordeaux
2,52,-10,Nantes


In [19]:
ger_old, ger_new=double_box_table('events_Germany.json', 'ger_shots')

<ipython-input-16-a3ed5dc79a08>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal']=goal
<ipython-input-16-a3ed5dc79a08>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['y']=y
<ipython-input-16-a3ed5dc79a08>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [4]:
ger_old

,points,goals_dif,name
1,84,64,Bayern M\u00fcnchen
5,63,16,Schalke 04
16,55,19,Hoffenheim
4,55,17,Borussia Dortmund
3,55,14,Bayer Leverkusen
17,53,4,RB Leipzig
2,51,0,Stuttgart
13,49,0,Eintracht Frankfurt
9,47,-5,Borussia M'gladbach
11,43,-3,Hertha BSC


In [5]:
ger_new

,points,goals_dif,name
1,85,77,Bayern M\u00fcnchen
5,68,23,Schalke 04
16,56,32,Hoffenheim
4,54,19,Borussia Dortmund
2,54,0,Stuttgart
9,51,-7,Borussia M'gladbach
3,50,18,Bayer Leverkusen
17,50,8,RB Leipzig
13,50,1,Eintracht Frankfurt
0,45,-2,Werder Bremen


In [20]:
ita_old, ita_new=double_box_table('events_Italy.json', 'ita_shots')

<ipython-input-16-a3ed5dc79a08>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal']=goal
<ipython-input-16-a3ed5dc79a08>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['y']=y
<ipython-input-16-a3ed5dc79a08>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [4]:
ita_old

,points,goals_dif,name
2,95,62,Juventus
13,91,48,Napoli
1,77,33,Roma
4,72,40,Lazio
3,72,36,Internazionale
0,64,14,Milan
9,60,18,Atalanta
11,57,8,Fiorentina
12,54,8,Torino
6,54,-4,Sampdoria


In [5]:
ita_new

,points,goals_dif,name
2,91,87,Juventus
13,91,67,Napoli
1,74,44,Roma
4,73,53,Lazio
0,70,20,Milan
3,65,46,Internazionale
9,55,15,Atalanta
12,54,12,Torino
11,54,7,Fiorentina
6,53,-12,Sampdoria


In [21]:
spa_old, spa_new=double_box_table('events_Spain.json', 'spa_shots')

<ipython-input-16-a3ed5dc79a08>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['goal']=goal
<ipython-input-16-a3ed5dc79a08>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots['y']=y
<ipython-input-16-a3ed5dc79a08>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [4]:
spa_old

,points,goals_dif,name
2,93,70,Barcelona
5,79,36,Atl\u00e9tico Madrid
1,76,50,Real Madrid
0,73,27,Valencia
7,61,7,Villarreal
9,60,-1,Real Betis
6,58,-9,Sevilla
15,55,9,Getafe
16,51,-6,Eibar
19,51,-9,Girona


In [5]:
spa_new

,points,goals_dif,name
2,96,101,Barcelona
5,83,47,Atl\u00e9tico Madrid
1,81,75,Real Madrid
0,72,35,Valencia
7,64,6,Villarreal
9,62,-8,Real Betis
6,56,-13,Sevilla
11,54,-2,Espanyol
15,53,15,Getafe
12,53,4,Celta de Vigo
